In [1]:
### looking at CRISPR KO gene length to determine if genes with upregulated paralogs are longer for Mellis et al. 2023
### Created by Madeline E Melzer on 20230816, Last edit by Madeline E Melzer on 20231126

In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
grn_nitc_path = "/Users/mem3579/Library/CloudStorage/OneDrive-NorthwesternUniversity/Arispe and Goyal Labs/transcriptionalCompensation/grn_nitc/"

fcDir = os.path.join(grn_nitc_path, "rnaseq/de_analysis/p_fc/FC2/tabular_format/")
pctUpregDir = os.path.join(grn_nitc_path, "rnaseq/de_analysis/p_fc/percent_upregulated/tabular_format/")
paralogDir = os.path.join(grn_nitc_path, "rnaseq/paralog_data/")
orthologDir = os.path.join(grn_nitc_path, "rnaseq/supp_analyses/orthologs/ortholog_data/")
outputDir = os.path.join(grn_nitc_path, "rnaseq/supp_analyses/length/data/")

np.random.seed(23)

In [6]:
# combining data files for paralogs

paralogFiles = [file for file in os.listdir(paralogDir) if file.endswith('.csv')]

paralogs = pd.DataFrame()

for file in paralogFiles:
    file_path = os.path.join(paralogDir, file)
    if os.path.getsize(file_path) == 0:
        print(f"Skipping empty file: {file}")
        continue
    data = pd.read_csv(file_path)
    data['dataset'] = os.path.splitext(file)[0]

    paralogs = pd.concat([paralogs, data], ignore_index=True)

combined_file_path = os.path.join(outputDir, 'combinedParalogs.csv')
#paralogs.to_csv(combined_file_path, index=False)


/var/folders/lj/t95zdh7s6q1728bs885tygr0f794qx/T/ipykernel_42570/4224509336.py:15: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  paralogs = pd.concat([paralogs, data], ignore_index=True)


In [7]:
# combining data files for fc

fcFiles = [file for file in os.listdir(fcDir) if file.endswith('.csv')]

fcs = pd.DataFrame()

for file in fcFiles:
    file_path = os.path.join(fcDir, file)
    if os.path.getsize(file_path) == 0:
        print(f"Skipping empty file: {file}")
        continue
    data = pd.read_csv(file_path)
    data['dataset'] = os.path.splitext(file)[0]

    fcs = pd.concat([fcs, data], ignore_index=True)

# Rename the "KO_Gene" column to "Gene" to match combinedParalogs
fcs.rename(columns={'KO_Gene': 'Gene'}, inplace=True)

combined_file_path = os.path.join(outputDir, 'combinedfcs.csv')
#fcs.to_csv(combined_file_path, index=False)


In [17]:
# combining all files for percent upregulated

pctUpregFiles = [file for file in os.listdir(pctUpregDir) if file.endswith('.csv')]

pctUpregs = pd.DataFrame()

for file in pctUpregFiles:
    file_path = os.path.join(pctUpregDir, file)
    if os.path.getsize(file_path) == 0:
        print(f"Skipping empty file: {file}")
        continue
    data = pd.read_csv(file_path)
    data = data[data['variable'] == 'Paralogs'] # Filter data to keep only rows where "Variable" is "Paralogs", not "Bootstrapped Genes"
    data['dataset'] = os.path.splitext(file)[0]

    pctUpregs = pd.concat([pctUpregs, data], ignore_index=True)

# Rename the "KO_Gene" column to "Gene" to match other dataframes
pctUpregs.rename(columns={'KO_Gene': 'Gene'}, inplace=True)

combined_file_path = os.path.join(outputDir, 'combinedPctUpregs.csv')
#pctUpregs.to_csv(combined_file_path, index=False)

In [8]:
# Combining the paralog, fold change (fc), and pctUpreg .csv files

# Load the combined CSV files
paralogs = pd.read_csv(os.path.join(outputDir, 'combinedParalogs.csv'))
fcs = pd.read_csv(os.path.join(outputDir, 'combinedfcs.csv'))
pctUpregs = pd.read_csv(os.path.join(outputDir, 'combinedPctUpregs.csv'))

# Create a subset of paralogs DataFrame
paralogs_subset = paralogs[['Gene', 'Paralog', 'ko_length', 'paralog_length']] # dont need the dataset column from paralogs. 

# Perform a left join based on "Gene" and "Paralog" columns
allData = pd.merge(fcs, paralogs_subset, how='left', on=['Gene', 'Paralog'])

# Adding in p-value of expression comparison
pctUpregs_subset = pctUpregs[['Gene', 'dataset', 'Percent Upregulated', 'p_value']] # Create a subset of pctUpregs DataFrame
allData = pd.merge(allData, pctUpregs_subset, how='left', on=['Gene', 'dataset']) # Perform a left join based on "Gene" column
allData.rename(columns={'p_value': 'pctUpreg_p_value'}, inplace=True) # Rename the 'P-value' column to 'pctUpreg_p_value' for expression comparison p-value
allData.reset_index(drop=True, inplace=True) # Reset the index and drop it so it doesnt interfere with dropping duplicate rows
allData.drop_duplicates(inplace=True) # Drop duplicate rows created by the join

# Save the joined data to a new CSV file
combined_file_path = os.path.join(outputDir, 'allData.csv')
#allData.to_csv(combined_file_path, index=False)




In [19]:
### checking dataframe for matching values in paper, Total CRISPR gene targets showing paralog upregulation = 16 (out of 73), Total upregulated paralogs = 57

# excluding secondary conditions not used in main analysis of Mellis et al. 2023
alt_sets = ['GSE92872-2', 'GSE145653-2', 'GSE161466-2', 'GSE161466-3', 'GSE161466-4', 'GSE175787-3', 'GSE175787-4', 'GSE175787-5', 'GSE175787-6']
mask = ~allData['dataset'].isin(alt_sets)
mainData = allData[mask]

combined_file_path = os.path.join(outputDir, 'mainData.csv')
#mainData.to_csv(combined_file_path, index=False)

print("Total number of unique CRISPR gene targets: ", mainData['Gene'].nunique())
print("Unique CRISPR gene targets: ", np.unique(mainData['Gene']))

# Filter rows according to cutoffs in Mellis et al. 2023
filtered_data = mainData[(mainData['pctUpreg_p_value'] < 0.1) & (mainData['padj'] < 0.05) & (mainData['FC2'] > 0.5)]

# Checking the number of unique genes
num_unique_genes_p = filtered_data['Gene'].nunique()
print(np.unique(filtered_data['Gene']))
print("Number of unique genes passing DESeq2 Filters: ", num_unique_genes_p)

# Checking the number of unique 'Gene-Paralog' pairs
filtered_data['Gene-Paralog'] = filtered_data['Gene'] + "-" + filtered_data['Paralog']
num_unique_pairs = filtered_data['Gene-Paralog'].nunique()
print("Number of unique 'Gene-Paralog' pairs: ", num_unique_pairs)


combined_file_path = os.path.join(outputDir, 'filteredData.csv')
#filtered_data.to_csv(combined_file_path, index=False)

Total number of unique CRISPR gene targets:  73
Unique CRISPR gene targets:  ['ASH1L' 'Actb' 'Actg1' 'Aff3' 'Apc' 'BUB1B' 'CMTR1' 'Cdk8' 'Csnk1a1'
 'Etv5' 'FOSL1' 'Fbxw7' 'Fermt2' 'Furin' 'Hprt' 'IKZF5' 'INO80' 'JUNB'
 'Jarid2' 'Jmjd1c' 'KDM1A' 'KLF6' 'KMT2A' 'L3mbtl3' 'LCK' 'Lmna' 'MBD2'
 'MTF1' 'Macf1' 'Mbd3' 'Msi2' 'Myc' 'Myo10' 'NSD1' 'Nes' 'Nmt1' 'Nsd1'
 'PLK1' 'POLK' 'PRPF4B' 'Pten' 'Pum1' 'RB1' 'REL' 'RUNX3' 'Raf1' 'RhoC'
 'SIX6' 'SMARCA4' 'SMC3' 'SOX10' 'SP1' 'SRF' 'STAG2' 'STK11' 'Smg7' 'TFAM'
 'TLR4' 'TP53' 'Tcf7l1' 'Tet1' 'Trim71' 'UBR5' 'Usp7' 'Usp9x' 'VEZF1'
 'WDR7' 'ZEB2' 'Zfp281' 'Zfp423' 'p50' 'p52' 'p65']
['ASH1L' 'Actb' 'Apc' 'KLF6' 'Lmna' 'Macf1' 'Myc' 'Nes' 'Nsd1' 'RB1'
 'RUNX3' 'SP1' 'TLR4' 'Trim71' 'ZEB2' 'Zfp281']
Number of unique genes passing DESeq2 Filters:  16
Number of unique 'Gene-Paralog' pairs:  57


/var/folders/lj/t95zdh7s6q1728bs885tygr0f794qx/T/ipykernel_31947/481864388.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['Gene-Paralog'] = filtered_data['Gene'] + "-" + filtered_data['Paralog']
